The **fastai** Deep Learning Library
===
> Documentation and tutorial: https://docs.fast.ai

Package versions and available devices:

In [1]:
import fastai
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"fastai version : {fastai.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"Number of GPUs : {torch.cuda.device_count()}")
print(f"Device         : {device}")

fastai version : 2.7.12
PyTorch version: 2.0.1
Number of GPUs : 0
Device         : cpu


In [2]:
from fastai.vision.all import *

In [3]:
%whos

Variable                           Type                          Data/Info
--------------------------------------------------------------------------
APScoreBinary                      function                      <function APScoreBinary at 0x7fc324125a60>
APScoreMulti                       function                      <function APScoreMulti at 0x7fc324133280>
AccumMetric                        type                          <class 'fastai.metrics.AccumMetric'>
ActivationStats                    type                          <class 'fastai.callback.hook.ActivationStats'>
Adam                               function                      <function Adam at 0x7fc3240f8280>
AdaptiveAvgPool                    function                      <function AdaptiveAvgPool at 0x7fc342eaeca0>
AdaptiveConcatPool1d               PrePostInitMeta               <class 'fastai.layers.AdaptiveConcatPool1d'>
AdaptiveConcatPool2d               PrePostInitMeta               <class 'fastai.layers.AdaptiveConcatP